In [ ]:
#импортируем данные с кэггла
import os
os.environ["KAGGLE_USERNAME"]="valeriianizolia"
os.environ['KAGGLE_KEY']='e537e2cce6a8da8f46f229a6f86662d9'
!kaggle competitions download -c recsys-iad-challenge

items.json.zip: Skipping, found more recently modified local copy (use --force to force download)
random_benchmark.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.json.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import numpy as np
import pandas as pd
import zipfile 
import string
import yaml
import ast
import json
from scipy.sparse import coo_matrix
from tqdm.auto import tqdm

In [ ]:
!unzip \*.zip

Archive:  items.json.zip
  inflating: items.json              

Archive:  random_benchmark.csv.zip
  inflating: random_benchmark.csv    

Archive:  train.json.zip
  inflating: train.json              

3 archives were successfully processed.


In [ ]:
%%time
#распаковка айтемс
items_list=[]
titles=[]
images=[]
content=[]
with tqdm(open('items.json')) as inf:
    for line in inf:
        item=json.loads(line)
        if isinstance(item['image'], float):
            item['image']=[0 for _ in range(96)]
            
        item['image']=np.array(item['image'])
        #del item['content']
        items_list.append(item)
        titles.append(item['title'])
        images.append(item['image'])
        content.append(item['content'])

items=pd.DataFrame(items_list).set_index('itemId')


CPU times: user 15.6 s, sys: 2.37 s, total: 17.9 s
Wall time: 17.5 s


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(lowercase=False, min_df = 10, max_df = 0.01)
tfidf_titles = vectorizer.fit_transform(titles)
tfidf_content = vectorizer.fit_transform(content)

In [ ]:
tfidf_titles

<328050x29762 sparse matrix of type '<class 'numpy.float64'>'
	with 1641340 stored elements in Compressed Sparse Row format>

In [ ]:
tfidf_content

<328050x239705 sparse matrix of type '<class 'numpy.float64'>'
	with 27815164 stored elements in Compressed Sparse Row format>

In [ ]:
from scipy.sparse import hstack
item_features = hstack([tfidf_content, tfidf_titles])

In [ ]:
#модель с картинками обучается очень долго, а изменение в качестве незначительное
#from scipy.sparse import *
#from scipy import *
#images = csr_matrix(np.array(images))

In [ ]:
#from scipy.sparse import hstack
#item_features = hstack([tfidf_titles, images])

In [ ]:
%%time
#распаковка train
user_list=[]
item_list=[]
target_list=[]
a=0
with tqdm(open('train.json')) as inf:
    for line in inf:
      a+=1
      train=json.loads(line)
      for key in train['trainRatings'].keys():
        item_list.append(key)
      for val in train['trainRatings'].values():
        target_list.append(val)
      for i in range(len(train['trainRatings'].keys())):
        user_list.append(train['userId'])


CPU times: user 46.1 s, sys: 11 s, total: 57.1 s
Wall time: 57.2 s


In [ ]:
#создадим спарс матрицу для более компактного хранения данных
row = np.array(user_list).astype(int)
col = np.array(item_list).astype(int)
data = np.array(target_list).astype(int)
coo = coo_matrix((data, (row, col)), shape=(a, len(items_list)))

In [ ]:
!pip install lightfm

     |████████████████████████████████| 307kB 2.8MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707608 sha256=6dc6a01600a7e029e50823f9760c2d674ebbd662568ee48ee0a095328ff71e96
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [ ]:
from lightfm import LightFM

model = LightFM(loss='bpr', no_components=30)

In [ ]:
%%time
model.fit(coo, num_threads=4, epochs = 100, verbose = True, item_features=item_features)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
CPU times: user 7h 8min 31s, sys: 3.18 s, total: 7h 8min 34s
Wall time: 1h 51min 22s


In [ ]:
random_benchmark = pd.read_csv("random_benchmark.csv")

In [ ]:
user_test = random_benchmark["userId"].values
item_test = random_benchmark['itemId'].values

In [ ]:
predictions = model.predict(user_test, item_test, num_threads=10, item_features=item_features)

In [ ]:
random_benchmark['predictions'] = predictions
random_benchmark

,userId,itemId,predictions
0,1,242249,-4.088578
1,1,117266,-4.107257
2,1,123441,-3.998820
3,1,148575,0.206054
4,1,155695,-1.121981
...,...,...,...
3018181,42975,4202,-3.747172
3018182,42975,111136,-1.727304
3018183,42975,195676,0.132611
3018184,42975,292604,0.516455


In [ ]:
random_sorted = random_benchmark.sort_values(['userId', 'predictions'], ascending=False)
random_sorted

,userId,itemId,predictions
3018026,42975,287643,3.643700
3018169,42975,7145,3.337272
3018008,42975,73069,3.303562
3018043,42975,222716,3.132034
3018067,42975,83665,2.810667
...,...,...,...
16,1,262104,-6.932975
318,1,16074,-6.965143
431,1,286918,-7.430414
385,1,132508,-7.684065


In [ ]:
del random_sorted['predictions']
random_sorted

,userId,itemId
3018026,42975,287643
3018169,42975,7145
3018008,42975,73069
3018043,42975,222716
3018067,42975,83665
...,...,...
16,1,262104
318,1,16074
431,1,286918
385,1,132508


In [ ]:
random_sorted.to_csv("submission_final.csv", index=False)